## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Here is the solution to the peer graded assignment on segmentation and clustering.

The first step is to import all needed libraries.
Note : Some are not really needed at this stage  but i have chosen to import it now. it will be needed in subsequent section of the project.

In [17]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

We have the url to the wikipedia page containing the relevant dat for analysis, hence we use the beautiful soup library for extraction of html tables

In [18]:
# Here we import the beautiful soup library to process the url and obtain the table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print(type(df))

<class 'list'>


We can see the data type of our output as a list

However, our daaframe is read as the whole first list position. So we can assign df as 'df[0]'

In [19]:
df = df[0]


In [20]:
df.head()`

SyntaxError: invalid syntax (<ipython-input-20-6b7eb21620cb>, line 1)

In [21]:
# Cross-Checking for the correct correct datatype formats
print (type(df))
print (df.shape)
df.dtypes

<class 'pandas.core.frame.DataFrame'>
(288, 3)


Postcode         object
Borough          object
Neighbourhood    object
dtype: object

Now it is obvious we have a pandas dataframe object

In [22]:
# Drop the rows where Borough is not assigned as stipulated in the assignment
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)

After dropping the rows with non-assigned values, we can see the shape of our dataframe and then reset index

In [23]:
# reset index
df.reset_index(drop=True, inplace=True)
df.shape

(211, 3)

Let's have a peep into  our dataframe

In [24]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Next task is to get the unique postcode and neighbour without repeating boroughs. We can  do this as follows

In [27]:
# To obtain unique postcode and Borough we use the groupby method and aggregate on neighbourhood
df =df.groupby(['Postcode','Borough'])['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()

# we iterate through all rows to change nieghbourhood with not assigned values to the borough values
for index, row in df.iterrows():
    if df.loc[index,'Neighbourhood'] == 'Not assigned':
        df.iloc[index][2]= df.iloc[index][1]# note column index 2==neighbourhood and 1==borough
    else: pass
    
    
df.head()
    

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We can use the shape method to obtain the number of rows and number of columns.

In [30]:
df.shape

(103, 3)